In [ ]:
#pip install psycopg2-binary

In [151]:
import sqlalchemy
import pandas as pd
import psycopg2
from sqlalchemy import create_engine, inspect

In [153]:
df_raw = pd.read_csv('data/silver_airline_quality_reviews.csv')
df_raw

,RowId,Airline Name,Review_Title,Review Date,Verified,Review,Top Review Image Url,Aircraft,Type Of Traveller,Seat Type,...,sentiment_score,img_resnet50_score,img_resnet50_label,img_blip_caption,img_clip_embedding,img_caption_clip_embedding,img_resnet50label_clip_embedding,img_vs_caption_similarity,img_caption_vs_resnetlabel_similarity,Overall_Rating
0,0,AB Aviation,"""pretty decent airline""",11th November 2019,True,Moroni to Moheli. Turned out to be a pretty ...,https://www.airlinequality.com/wp-content/uplo...,NaN,Solo Leisure,Economy Class,...,0.918486,5.190325,military aircraft,a man taking a picture of a plane,"tensor([[ 3.5914e-02, -2.8891e-02, 4.5510e-03...","tensor([[ 1.0825e-02, 8.4288e-03, -3.1653e-02...","tensor([[ 9.1185e-03, -1.0847e-02, 1.6327e-02...",0.251976,0.738985,9
1,1,AB Aviation,"""Not a good airline""",25th June 2019,True,Moroni to Anjouan. It is a very small airline...,NaN,E120,Solo Leisure,Economy Class,...,0.569241,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,2,AB Aviation,"""flight was fortunately short""",25th June 2019,True,Anjouan to Dzaoudzi. A very small airline an...,NaN,Embraer E120,Solo Leisure,Economy Class,...,0.542810,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,3,Adria Airways,"""I will never fly again with Adria""",28th September 2019,False,Please do a favor yourself and do not fly wi...,NaN,NaN,Solo Leisure,Economy Class,...,0.903330,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4,4,Adria Airways,"""it ruined our last days of holidays""",24th September 2019,True,Do not book a flight with this airline! My fr...,NaN,NaN,Couple Leisure,Economy Class,...,0.902472,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24349,24349,ZIPAIR,"""customer service is terrible""",5th July 2022,False,Bangkok to Tokyo. I’ve flown many low cost ai...,NaN,NaN,Couple Leisure,Economy Class,...,0.842372,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
24350,24350,ZIPAIR,"""Avoid at all costs""",1st June 2022,True,Avoid at all costs. I booked flights to go f...,NaN,NaN,Solo Leisure,Economy Class,...,0.836562,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
24351,24351,ZIPAIR,"""Will not recommend to anyone""",31st May 2022,True,Flight was leaving at 23.15 and after an hou...,NaN,NaN,Business,Economy Class,...,0.797939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
24352,24352,ZIPAIR,"""It was immaculately clean""",23rd May 2022,True,Zipair is JAL’s budget airline. They don’t ha...,NaN,Dreamliner,Business,Business Class,...,0.654033,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6


In [157]:
# Data Preprocessing
#needed_columns = [c for c in df_raw.columns if c != 'Unnamed: 0']
needed_columns = ['RowId',
 'Airline Name',
 'Review_Title',
 'Review Date',
 'Verified',
 'Review',
 'Top Review Image Url',
 'Aircraft',
 'Type Of Traveller',
 'Seat Type',
 'Route',
 'Date Flown',
 'Seat Comfort',
 'Cabin Staff Service',
 'Food & Beverages',
 'Ground Service',
 'Inflight Entertainment',
 'Wifi & Connectivity',
 'Value For Money',
 'Recommended',
 'sentiment_label',
 'Overall_Rating',
 'Id']

In [159]:
df = df_raw[needed_columns].copy()
df['Review Date'] = pd.to_datetime(df['Review Date'], format="mixed")
df['Date Flown'] = pd.to_datetime(df['Date Flown'], format="%b-%y")
df

,RowId,Airline Name,Review_Title,Review Date,Verified,Review,Top Review Image Url,Aircraft,Type Of Traveller,Seat Type,...,Cabin Staff Service,Food & Beverages,Ground Service,Inflight Entertainment,Wifi & Connectivity,Value For Money,Recommended,sentiment_label,Overall_Rating,Id
0,0,AB Aviation,"""pretty decent airline""",2019-11-11,True,Moroni to Moheli. Turned out to be a pretty ...,https://www.airlinequality.com/wp-content/uplo...,NaN,Solo Leisure,Economy Class,...,5.0,4.0,4.0,NaN,NaN,3.0,yes,positive,9,5cb0f79de5c9e67eb8b30829f79c4954df18ccb36ac930...
1,1,AB Aviation,"""Not a good airline""",2019-06-25,True,Moroni to Anjouan. It is a very small airline...,NaN,E120,Solo Leisure,Economy Class,...,2.0,1.0,1.0,NaN,NaN,2.0,no,negative,1,8e6c32573411848ebb03db8611ba048ed404efb10e3024...
2,2,AB Aviation,"""flight was fortunately short""",2019-06-25,True,Anjouan to Dzaoudzi. A very small airline an...,NaN,Embraer E120,Solo Leisure,Economy Class,...,1.0,1.0,1.0,NaN,NaN,2.0,no,neutral,1,87eb30af6644384e7f6b999af1496654808d0b20fdbf41...
3,3,Adria Airways,"""I will never fly again with Adria""",2019-09-28,False,Please do a favor yourself and do not fly wi...,NaN,NaN,Solo Leisure,Economy Class,...,1.0,NaN,1.0,NaN,NaN,1.0,no,negative,1,7f7f715210dce748407158608b5d19354598b50d5019ad...
4,4,Adria Airways,"""it ruined our last days of holidays""",2019-09-24,True,Do not book a flight with this airline! My fr...,NaN,NaN,Couple Leisure,Economy Class,...,1.0,1.0,1.0,1.0,1.0,1.0,no,negative,1,7da7e8cd22cf9233f1427953365c03263d0951b930f3d4...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24349,24349,ZIPAIR,"""customer service is terrible""",2022-07-05,False,Bangkok to Tokyo. I’ve flown many low cost ai...,NaN,NaN,Couple Leisure,Economy Class,...,1.0,NaN,1.0,1.0,1.0,1.0,no,negative,1,95534770c2b79accb9aeb5347f2e3ece086849d1118dcc...
24350,24350,ZIPAIR,"""Avoid at all costs""",2022-06-01,True,Avoid at all costs. I booked flights to go f...,NaN,NaN,Solo Leisure,Economy Class,...,NaN,NaN,NaN,NaN,NaN,1.0,no,negative,1,ffba6278f93ca5b66d2b6516719645cc30455f97e5563b...
24351,24351,ZIPAIR,"""Will not recommend to anyone""",2022-05-31,True,Flight was leaving at 23.15 and after an hou...,NaN,NaN,Business,Economy Class,...,4.0,2.0,1.0,NaN,NaN,2.0,no,negative,3,b7dcd58ea6a1975f78208e032290f85e6280224fbe70bb...
24352,24352,ZIPAIR,"""It was immaculately clean""",2022-05-23,True,Zipair is JAL’s budget airline. They don’t ha...,NaN,Dreamliner,Business,Business Class,...,4.0,3.0,1.0,2.0,5.0,5.0,yes,neutral,6,f0cf15a6df3e3ba4781bec2ab48e7aed5b8683d64cc24d...


In [161]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24354 entries, 0 to 24353
Data columns (total 23 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   RowId                   24354 non-null  int64         
 1   Airline Name            24354 non-null  object        
 2   Review_Title            24354 non-null  object        
 3   Review Date             24354 non-null  datetime64[ns]
 4   Verified                24354 non-null  bool          
 5   Review                  24354 non-null  object        
 6   Top Review Image Url    1897 non-null   object        
 7   Aircraft                7727 non-null   object        
 8   Type Of Traveller       20759 non-null  object        
 9   Seat Type               23288 non-null  object        
 10  Route                   20679 non-null  object        
 11  Date Flown              20743 non-null  datetime64[ns]
 12  Seat Comfort            20288 non-null  float6

In [163]:
#Get DB credentials
from dotenv import load_dotenv
import os

from dotenv import load_dotenv
from pathlib import Path
import os

dotenv_path = Path('db_credentials.env')
load_dotenv(dotenv_path=dotenv_path)

DB_HOST = os.getenv('DB_HOST')
DB_PORT = os.getenv('DB_PORT')
DB_NAME = os.getenv('DB_NAME')
DB_USER = os.getenv('DB_USER')
DB_PASSWORD = os.getenv('DB_PASSWORD')
DATABRICKS_TOKEN = os.getenv('DATABRICKS_TOKEN')
DATABRICKS_INSTANCE = os.getenv('DATABRICKS_INSTANCE')

In [165]:
#Connect to RDS
conf ={
    'host':DB_HOST,
    'port':DB_PORT,
    'database':DB_NAME,
    'user':DB_USER,
    'password':DB_PASSWORD
}
engine = create_engine("postgresql://{user}:{password}@{host}:{port}/{user}".format(**conf))

df.to_sql('staging_airline_quality_reviews', engine, index=False, if_exists='replace')

354

In [166]:
'''
#inspect what tables are on the RDS
inspector = inspect(engine)
tables = inspector.get_table_names()
print(tables)
'''

'\n#inspect what tables are on the RDS\ninspector = inspect(engine)\ntables = inspector.get_table_names()\nprint(tables)\n'

In [145]:
'''
#take a look at the table
sql="""
SELECT COUNT(*) FROM silver_airline_quality_reviews
"""
df_back = pd.read_sql_query(sql, engine)
print(df_back)
'''

'\n#take a look at the table\nsql="""\nSELECT COUNT(*) FROM silver_airline_quality_reviews\n"""\ndf_back = pd.read_sql_query(sql, engine)\nprint(df_back)\n'

In [147]:
'''# RUN THIS INITIALLY
try:
    # Connect to the PostgreSQL database
    connection = psycopg2.connect(
        dbname=DB_NAME,
        user=DB_USER,
        password=DB_PASSWORD,
        host=DB_HOST,
        port=DB_PORT
    )
    
    cursor = connection.cursor()
    
    cursor.execute("""
                DROP TABLE IF EXISTS "silver_airline_quality_reviews";
                """)
            
    connection.commit()
    print(f"Table 'silver_airline_quality_reviews' dropped successfully.")



    cursor = connection.cursor()
    
    # Create the users table
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS silver_airline_quality_reviews (
    	"Id" varchar(64) NOT NULL, 
        "RowId" int8 NULL,
    	"Airline Name" varchar(255) NULL,
    	"Review_Title" varchar(1000) NULL,
    	"Review Date" timestamp NULL,
    	"Verified" bool NULL,
    	"Review" text NULL,
    	"Top Review Image Url" text NULL,
    	"Aircraft" varchar(255) NULL,
    	"Type Of Traveller" varchar(255) NULL,
    	"Seat Type" varchar(255) NULL,
    	"Route" varchar(1000) NULL,
    	"Date Flown" timestamp NULL,
    	"Seat Comfort" float8 NULL,
    	"Cabin Staff Service" float8 NULL,
    	"Food & Beverages" float8 NULL,
    	"Ground Service" float8 NULL,
    	"Inflight Entertainment" float8 NULL,
    	"Wifi & Connectivity" float8 NULL,
    	"Value For Money" float8 NULL,
    	"Recommended" bool NULL,
    	sentiment_label varchar(50) NULL,
    	"Overall_Rating" float8 NULL,
        is_lost_luggage_flag bool NULL,
        created_by varchar(100) NOT NULL,
        created_date timestamp NOT NULL DEFAULT NOW(),
        lastmodified_by varchar(100) NOT NULL,
        lastmodified_date timestamp NOT NULL DEFAULT NOW()   
    );
    """)
    connection.commit()
    print("Table 'silver_kaggle_airline_reviews' created successfully.")

except (Exception, psycopg2.Error) as error:
    print("Error during drop-create operation:", error)
finally:
    if connection:
        cursor.close()
        connection.close()
'''

Table 'silver_airline_quality_reviews' dropped successfully.
Table 'silver_kaggle_airline_reviews' created successfully.


In [169]:
try:
    # Connect to the PostgreSQL database
    connection = psycopg2.connect(
        dbname=DB_NAME,
        user=DB_USER,
        password=DB_PASSWORD,
        host=DB_HOST,
        port=DB_PORT
    )

    cursor = connection.cursor()
    
    # Create the users table
    cursor.execute("""
        INSERT INTO silver_airline_quality_reviews 
        (
        	"Id",
            "RowId",
        	"Airline Name",
        	"Review_Title",
        	"Review Date",
        	"Verified",
        	"Review",
        	"Top Review Image Url",
        	"Aircraft",
        	"Type Of Traveller",
        	"Seat Type",
        	"Route",
        	"Date Flown",
        	"Seat Comfort",
        	"Cabin Staff Service",
        	"Food & Beverages",
        	"Ground Service",
        	"Inflight Entertainment",
        	"Wifi & Connectivity",
        	"Value For Money",
        	"Recommended",
        	sentiment_label,
        	"Overall_Rating",
            is_lost_luggage_flag,
            created_by,
            created_date,
            lastmodified_by,
            lastmodified_date           
        )
        SELECT
        	"Id",
            "RowId",
        	"Airline Name",
        	"Review_Title",
        	"Review Date",
        	"Verified",
        	"Review",
        	"Top Review Image Url",
        	"Aircraft",
        	"Type Of Traveller",
        	"Seat Type",
        	"Route",
        	"Date Flown",
        	"Seat Comfort",
        	"Cabin Staff Service",
        	"Food & Beverages",
        	"Ground Service",
        	"Inflight Entertainment",
        	"Wifi & Connectivity",
        	"Value For Money",
        	CAST("Recommended" AS bool) AS "Recommended",
        	sentiment_label,
            CASE WHEN "Overall_Rating" = 'n' THEN NULL ELSE CAST("Overall_Rating" AS float8) END AS rating_overall,
            NULL AS is_lost_luggage_flag,
            'From_Kaggle' AS created_by,
            NOW() AS created_date,
            'From_Kaggle' AS lastmodified_by,
            NOW() AS lastmodified_date
        FROM staging_airline_quality_reviews
    """)

    cursor.execute("""
                DROP TABLE IF EXISTS staging_airline_quality_reviews;
                """)
    
    connection.commit()
    print('Inserted to "silver_airline_quality_reviews" successfully')


except (Exception, psycopg2.Error) as error:
    print("Error during insert operation:", error)
finally:
    if connection:
        cursor.close()
        connection.close()

Inserted to "silver_airline_quality_reviews" successfully


#### Upload Dataframe to Databricks volume

In [8]:
import requests

In [16]:
#Using Databricks Upload a file API
#https://docs.databricks.com/api/workspace/files/upload
databricks_instance = DATABRICKS_INSTANCE
local_path = 'data/silver_airline_quality_reviews.csv'
remote_path = '/Volumes/workspace/data608/data608_volume/silver_airline_quality_reviews.csv'
headers = {'Authorization' : f"Bearer {DATABRICKS_TOKEN}"}
#print(headers)

with open(local_path, 'rb') as f:
    r = requests.put(
      f"https://{databricks_instance}/api/2.0/fs/files{remote_path}?overwrite=true",
      headers=headers,
      data=f)
    r.raise_for_status()

print(f"Uploaded '{local_path}' to '{remote_path}' on Databricks workspace {databricks_instance}")

Uploaded 'data/Airline_review.csv' to '/Volumes/workspace/data608/data608_volume/Kaggle_airline_reviews.csv' on Databricks workspace dbc-daf0509f-1d8b.cloud.databricks.com
